In [1]:
import sys
sys.path.append('src/')
sys.path.append('scripts/')

In [2]:
import os
import torchvision.datasets as datasets
import torch
import torch
import logging
from pythae.models import LVAE_IAF, LVAE_IAF_Config, AutoModel
from pythae.trainers import BaseTrainerConfig, BaseTrainer
import argparse
import numpy as np

from utils import Encoder_ColorMNIST, Encoder_Chairs, Decoder_ColorMNIST, Decoder_Chairs, Encoder_Faces, Decoder_Faces, My_Dataset, My_MaskedDataset, make_batched_masks

In [4]:
!ls

MANIFEST.in       experiments.ipynb pyproject.toml    setup.cfg
README.md         my_data           requirements.txt  setup.py
data_folders      plots             scripts           src


In [5]:
train_data = torch.load(os.path.join('my_data/sprites/Sprites_train.pt'))[:-1000]
eval_data = torch.load(os.path.join('my_data/sprites/Sprites_train.pt'), map_location="cpu")[-1000:]
test_data = torch.load(os.path.join('my_data/sprites/Sprites_test.pt'), map_location="cpu")

In [6]:
input_dim = (3, 64, 64)
latent_dim = 16
beta = 1
n_hidden_in_made = 3
n_made_blocks = 2
warmup = 10
context_dim = None
prior = 'vamp'
posterior = 'iaf'
vamp_number_components= 500
linear_scheduling_steps = 10
num_epochs = 400
batch_size = 64
learning_rate=  1e-3 
steps_saving = None
steps_predict = 100
shuffle_data = True


In [7]:
config = LVAE_IAF_Config(
    input_dim=input_dim,
    n_obs_per_ind=train_data.shape[1],
    latent_dim=latent_dim,
    beta=beta,
    n_hidden_in_made=n_hidden_in_made,
    n_made_blocks=n_made_blocks,
    warmup=warmup,
    context_dim=context_dim,
    prior=prior,
    posterior=posterior,
    vamp_number_components=vamp_number_components,
    linear_scheduling_steps=linear_scheduling_steps
)

training_config = BaseTrainerConfig(
        num_epochs=num_epochs,
        learning_rate=learning_rate,
        batch_size=batch_size,
        steps_saving=steps_saving,
        steps_predict=steps_predict,
        shuffle=shuffle_data
    )

In [8]:
encoder = Encoder_Chairs(config)
decoder = Decoder_Chairs(config)

model = LVAE_IAF(config, encoder, decoder).to('cpu')

### Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=training_config.learning_rate, eps=1e-4)

### Scheduler
scheduler = torch.optim.lr_scheduler.MultiStepLR(
    optimizer,
    milestones=[50, 100, 125, 150],
    gamma=0.5,
    verbose=True
)

/opt/miniconda3/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [9]:
train_seq_mask = torch.ones(train_data.shape[:2], requires_grad=False).type(torch.bool)
eval_seq_mask = torch.ones(eval_data.shape[:2], requires_grad=False).type(torch.bool)
test_seq_mask = torch.ones(test_data.shape[:2], requires_grad=False).type(torch.bool)
train_pix_mask = torch.ones_like(train_data, requires_grad=False).type(torch.bool)
eval_pix_mask = torch.ones_like(eval_data, requires_grad=False).type(torch.bool)
test_pix_mask = torch.ones_like(test_data, requires_grad=False).type(torch.bool)

train_dataset = My_MaskedDataset(train_data, train_seq_mask, train_pix_mask)
eval_dataset = My_MaskedDataset(eval_data, eval_seq_mask, eval_pix_mask)
test_dataset = My_MaskedDataset(test_data, test_seq_mask, test_pix_mask)




In [10]:
trainer = BaseTrainer(
            model=model,
            train_dataset=train_dataset,
            eval_dataset=eval_dataset,
            training_config=training_config,
            optimizer=optimizer,
            scheduler=scheduler,
        )

trainer.train()

Created dummy_output_dir folder since did not exist.

Model passed sanity check !

Created dummy_output_dir/LVAE_IAF_training_2024-04-30_13-28-35. 
Training config, checkpoints and final model will be saved here.

Successfully launched training !



Training of epoch 1/400:   0%|          | 0/125 [00:00<?, ?batch/s]